In [1]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.9 MB/s eta 0:00:00


In [2]:
from PIL import Image
import requests
from peft import get_peft_model, LoraConfig
from transformers import AutoProcessor, BlipForQuestionAnswering

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 384672572 || all params: 384672572 || trainable%: 100.0


In [4]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# # training
# text = "How many cats are in the picture?"
# label = "2"
# inputs = processor(images=image, text=text, return_tensors="pt")
# labels = processor(text=label, return_tensors="pt").input_ids

# inputs["labels"] = labels
# outputs = model(**inputs)
# loss = outputs.loss
# loss.backward()

# inference
text = "How many cats are in the picture?"
inputs = processor(images=image, text=text, return_tensors="pt")
outputs = model.generate(**inputs)
print(processor.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


2


In [5]:
# Define LoRA configuration for the BLIP VQA model
config = LoraConfig(
    r=16,  # Rank of LoRA, adjust as needed
    lora_alpha=32,  # Scaling factor, adjust as needed
    lora_dropout=0.1,  # Dropout for LoRA layers, adjust as needed
    bias="none",  # Bias configuration for LoRA layers
    target_modules=["query", "value"]  # Target modules in the Transformer to apply LoRA
 )

# Acquire the LoRA-adapted model
peft_model = get_peft_model(model, config)
print_trainable_parameters(peft_model)

trainable params: 2359296 || all params: 387031868 || trainable%: 0.6095870120958619


In [6]:
url1 = "http://images.cocodataset.org/val2017/000000039769.jpg"
image1 = Image.open(requests.get(url, stream=True).raw)

# training
text1 = "How many cats are in the picture?"
label1 = "2"
inputs1 = processor(images=image1, text=text1, return_tensors="pt")
labels1 = processor(text=label1, return_tensors="pt").input_ids

inputs1["labels"] = labels1
outputs1 = peft_model(**inputs1)
loss1 = outputs1.loss
loss1.backward()

# inference
text1 = "How many cats are in the picture?"
inputs1 = processor(images=image1, text=text1, return_tensors="pt")
outputs1 = peft_model.generate(**inputs1)
print(processor.decode(outputs1[0], skip_special_tokens=True))

2
